# CAVEAT
The following instruction **does not work for 3D plots**:
```
ax.set_aspect(aspect = 'equal')
```

# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from skimage import io

# Allow the interactive rotation of 3D scatter plots in jupyter notebook
import sys    
import os    
file_name =  os.path.basename(sys.argv[0])
#print(file_name == 'ipykernel_launcher.py') # This basicaly asks whether this file is a jupyter notebook?
if __name__ == "__main__":
    if file_name == 'ipykernel_launcher.py': # Run only in .ipynb, not in exported .py scripts
        get_ipython().run_line_magic('matplotlib', 'notebook') # Equivalent to ''%matplotlib notebook', but it is also understood by .py scripts
        
import functions_data_processing    
from functions_data_processing import load_camera_params, get_variables_from_vertex_full_Dataframe, load_faces
import functions_plot
from functions_plot import plot_camera_and_vertices

In [ ]:
print('Image size:')
functions_data_processing.get_picture_size(verbose=1)

# 3rd DataSet - RendersTowelWall3 (20/11/18)
Towel Wall hang from different places, where a subsample of vertices is chosen.Subsampled dataset

# Choose number of vertices in each direction

In [ ]:
submesh_num_vertices_horizontal = 8
submesh_num_vertices_vertical = 12

from submesh import submesh_idx_from_num_vertices_in_each_direction
submesh_idx = submesh_idx_from_num_vertices_in_each_direction(submesh_num_vertices_vertical = submesh_num_vertices_vertical,
                                                              submesh_num_vertices_horizontal = submesh_num_vertices_horizontal)
print(submesh_idx)

In [ ]:
###
### 3rd DataSet - RendersTowelWall3
###
# Choose sequence, animation frame and order of vertices
sequence_name = 'TowelWall'
dataset_number = '3'
group_number = '001'
animation_frame = '00022'
reordered=1

# Examples for which uv coordinates seem to perfectly match RGB
# Example 1
group_number = '005'
animation_frame = '00004'
# Example 2
group_number = '002'
animation_frame = '00031'

# Examples for which uv coordinates don't seem to perfectly match RGB
# Example 1
group_number = '002'
animation_frame = '00021'
# Example 2
group_number = '010'
animation_frame = '00021'
# Example 3
group_number = '001'
animation_frame = '00001'

# Load camera parameters
variables = load_camera_params(sequence_name = sequence_name, dataset_number = dataset_number)
RT_matrix = variables['RT_matrix']
RT_extended = variables['RT_extended']
camera_worldCoord_x = variables['camera_worldCoord_x']
camera_worldCoord_y = variables['camera_worldCoord_y']
camera_worldCoord_z = variables['camera_worldCoord_z']
Intrinsic_matrix = variables['Intrinsic_matrix']
Camera_proj_matrix = variables['Camera_proj_matrix']

# Load vertex data
variables = get_variables_from_vertex_full_Dataframe(sequence_name=sequence_name, dataset_number=dataset_number, 
                                                     group_number=group_number, animation_frame=animation_frame,
                                                     RT_extended=RT_extended, reordered=reordered,
                                                     submesh_idx=submesh_idx, verbose=1)

occlusion_mask_values = variables['occlusion_mask_values']
u = variables['u']
v = variables['v']
u_visible = variables['u_visible']
v_visible = variables['v_visible']
u_occluded = variables['u_occluded']
v_occluded = variables['v_occluded']
X_world = variables['X_world']
Y_world = variables['Y_world']
Z_world = variables['Z_world']
X_camera = variables['X_camera']
Y_camera = variables['Y_camera']
Z_camera = variables['Z_camera']
nX_world = variables['nX_world']
nY_world = variables['nY_world']
nZ_world = variables['nZ_world']

# Load face data
faces = load_faces(sequence_name = sequence_name, dataset_number = dataset_number, verbose=1, reordered = reordered)

## Showing the 2D animation frame

In [ ]:
functions_plot.show_2D_animation_frame(sequence_name, dataset_number, group_number, animation_frame)

## 3D scatter plot of the vertices
(in world coordinates)

In [ ]:
xmin, ymin, zmin, xmax, ymax, zmax= functions_plot.plot_3D_vertices(X=X_world, Y=Y_world, Z=Z_world, sequence_name=sequence_name,
                        dataset_number=dataset_number, 
                        group_number=group_number, 
                        animation_frame=animation_frame)

## Plot the normal vectors to the mesh at the vertices
You can choose between world/camera coordinates
and whether to plot the camera position as well.

In [ ]:
functions_plot.plot_normal_vectors(sequence_name=sequence_name,
                        dataset_number=dataset_number, 
                        group_number=group_number, 
                        animation_frame=animation_frame, 
                        X=X_world, Y=Y_world, Z=Z_world,
                        nX=nX_world, nY=nY_world, nZ=nZ_world,
                        occlusion_mask_values=occlusion_mask_values,
                        camera_x=camera_worldCoord_x,
                        camera_y=camera_worldCoord_y,
                        camera_z=camera_worldCoord_z,
                        xmin=xmin, ymin=ymin, zmin=zmin, xmax=xmax, ymax=ymax, zmax=zmax,
                        plot_camera=1, swap_axes=0)

## 3D scatter plot of the vertices coloured by visibility
(in world coordinates)

In [ ]:
# 3D scatter plot of the vertices coloured by visibility
# (with legend)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

unique = list(set(occlusion_mask_values))
#colors = [plt.cm.jet(float(i)/max(unique)) for i in unique]
colors= ['b', 'y']
for i, un in enumerate(unique):
    xi = [X_world[j] for j  in range(len(X_world)) if occlusion_mask_values[j] == un]
    yi = [Y_world[j] for j  in range(len(Y_world)) if occlusion_mask_values[j] == un]
    zi = [Z_world[j] for j  in range(len(Z_world)) if occlusion_mask_values[j] == un]

    ax.scatter(xi, yi, zi, c=colors[i], marker='o', s=50)

ax.legend(['Visible', 'occluded'], loc=3)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.title('Vertex visibility of mesh in ' + sequence_name + '_' + dataset_number + '\nGroup ' + group_number + ' - Animation frame ' + animation_frame)

ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])
ax.set_zlim([zmin, zmax])

#ax.set_aspect(aspect = 'equal')

plt.show()

# fig.savefig('VisualizationTest/vertex_visibility_' + sequence_name + dataset_number + '_Group' + group_number + '_frame' + animation_frame + '.png')

## Plot camera position and 3D scatter plot the vertices coloured by visibility
(in world coordinates)

In [ ]:
# 3D scatter plot of the vertices coloured by visibility
# Including camera position

# plot_camera_and_vertices(X=X_world, Y=Y_world, Z=Z_world, vertex_size=50)

plot_camera_and_vertices(sequence_name=sequence_name,
                             dataset_number=dataset_number, 
                             group_number=group_number, 
                             animation_frame=animation_frame, 
                             X=X_world, Y=Y_world, Z=Z_world,
                             occlusion_mask_values=occlusion_mask_values,
                             camera_x=camera_worldCoord_x,
                             camera_y=camera_worldCoord_y,
                             camera_z=camera_worldCoord_z,
                             xmin=xmin, ymin=ymin, zmin=zmin, xmax=xmax, ymax=ymax, zmax=zmax,
                             swap_axes=0,
                             vertex_size=50)

## 2D scatter plot of the pixel location of each vertex
Using both visible and occluded vertices

In [ ]:
# 2D scatter plot of the pixel location of each vertex
# Using both visible and occluded vertices

functions_plot.plot_uv_on_RGB(u_visible=u_visible, v_visible=v_visible, 
                                      u_occluded=u_occluded, v_occluded=v_occluded,
                                      sequence_name=sequence_name, dataset_number=dataset_number, 
                                      group_number=group_number, animation_frame=animation_frame)

## 2D scatter plot of the pixel location of each vertex
Using only visible vertices

In [ ]:
# 2D scatter plot of the pixel location of each vertex
# Using only visible vertices

fig = plt.figure()
ax = fig.add_subplot(111)

ax.scatter(u_visible, v_visible, c='b', marker='o', s=50)

plt.gca().invert_yaxis() # the origin of the (u, v) coordinates is top-left

ax.set_xlabel('u')
ax.set_ylabel('v')

plt.title('Pixel location of each visible vertex\n' + sequence_name + '_' + dataset_number + ' - Group ' + group_number + ' - Animation frame ' + animation_frame)

ax.set_aspect(aspect = 'equal')

plt.show()

fig.savefig('VisualizationTest/pixelLocationOfVisibleVertices_' + sequence_name + dataset_number + '_Group' + group_number + '_frame' + animation_frame + '.png')

## Plot RGB and landmaks

In [ ]:
functions_plot.plot_RGB_and_landmarks(u_visible=u_visible, v_visible=v_visible, 
                                      u_occluded=u_occluded, v_occluded=v_occluded,
                                      sequence_name=sequence_name, dataset_number=dataset_number, 
                                      group_number=group_number, animation_frame=animation_frame)

# Normalize 0<=u,v<=1

In [ ]:
print(u)
u, v = functions_data_processing.normalize_uv_01(u=u, v=v)
print(u)
u, v = functions_data_processing.unnormalize_uv_01(u=u, v=v)
print(u)

In [ ]:
filename = 'Renders' + sequence_name + dataset_number + '/Group.' + group_number 
filename += '/vertices_' + animation_frame
if reordered == 1:
    filename+= '_reordered'
filename+='.txt'
f = open(filename, 'r')
line1 = f.readline()
df_vertices_all_data = pd.read_csv(f, sep = ' ',  names = line1.replace('# ', '').split())

#
# Select the submesh only
#
if submesh_idx!=None:
    df_vertices_all_data = df_vertices_all_data.ix[submesh_idx]

# Normalize
uv = df_vertices_all_data[['u', 'v']].values
uv_original=uv
uv = functions_data_processing.normalize_uv_01(uv=uv)

# Unnormalize
uv = functions_data_processing.unnormalize_uv_01(uv=uv)
print(uv_original==uv)